In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
store = Store(os.getenv('localappdata') + r'\Kaspad\kaspa-mainnet\datadir2')

In [3]:
store.load_recent_blocks()

In [4]:
header_fields = ['timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty']
block_fields = ['pubkey_script', 'kaspad_version', 'miner_version']
count_fields = [] # ['num_parents', 'num_children', 'num_blues', 'num_reds']
frames = store.load_data(header_fields=header_fields, block_fields=block_fields, count_fields=count_fields)

  0%|          | 0/3577 [00:00<?, ?it/s]

In [5]:
frames.keys()

dict_keys(['hash', 'timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty', 'pubkey_script', 'kaspad_version', 'miner_version'])

In [6]:
df = pd.DataFrame(frames).set_index('hash')

In [7]:
pp_time = store.get_header_data(store.pruning_point()).timeInMilliseconds
pp_time, len(df)

(1650842599892, 3577)

In [8]:
df['kaspad_version'].value_counts()

0.12.0     1792
0.11.17    1755
0.11.16      10
0.11.15       7
0.11.14       7
unknown       6
Name: kaspad_version, dtype: int64

In [9]:
(df['kaspad_version'] + '/' + df['miner_version']).value_counts(normalize=True)

0.11.17/                            0.487559
0.12.0/striker                      0.382723
0.12.0/ca.acc-pool.pw               0.043332
0.12.0/ge.acc-pool.pw               0.026838
0.12.0/BzMiner-v8.1.4               0.019849
0.12.0/                             0.010344
0.12.0/0.2.1-GPU-0.6/2204202326     0.009226
0.11.16/0.2.1-GPU-0.5/2204021842    0.002796
0.11.15/BzMiner-v8.1.4              0.001957
0.11.14/unknown                     0.001957
0.11.17/hk.acc-pool.pw              0.001677
unknown/unknown                     0.001677
0.12.0/0.2.1-GPU-0.5/2204021837     0.001398
0.12.0/BzMiner-v9.0.0b4             0.001118
0.12.0/0.2.1-GPU-0.5/2204061443     0.000839
0.12.0/0.2.1-GPU-0.6/2204202030     0.000839
0.12.0/0.2.1-GPU-0.6/2204202324     0.000839
0.12.0/0.2.1-GPU-0.5/2204081349     0.000839
0.11.17/BzMiner-v8.2.0b1            0.000559
0.12.0/BzMiner-v9.0.0               0.000559
0.12.0/ru.acc-pool.pw               0.000559
0.12.0/0.2.1-GPU-0.6/2204181211     0.000559
0.12.0/0.2

In [10]:
# df["address"] = df["pubkey_script"].apply(kbech32.toAddress)

In [11]:
store.close()